## Creating a class for representing multiple qubits and their operations 
Each individual qubit can be represented as a two dimensional vector with scalars from the feild of complex numbers. Formally this means that each qubit can be represented in the form $\alpha|0\rangle + \beta|1\rangle$ where $\alpha$ and $\beta$ are complex numbers such that $|\alpha|^2 + |\beta|^2 = 1$. Note that $|0\rangle = \begin{bmatrix}1 \\ 0 \end{bmatrix}$ and $|1\rangle = \begin{bmatrix}0 \\ 1 \end{bmatrix}$.
<br><br><br><br><br>
But representing qubits alone is not very useful we need to be able to represent multiple qubits and their interactions. We can take the vector representation of two qubits and represent both of them using a single vector with the kronecker product. The kronecker product between two vectors is defined as follows:
$
\begin{bmatrix}
a_1 \\
a_2
\end{bmatrix} \otimes 
\begin{bmatrix}
b_1 \\
b_2 
\end{bmatrix} = \begin{bmatrix}
a_1 b_1 \\
a_1 b_2 \\
a_2 b_1 \\
a_2 b_2 \\
\end{bmatrix}$. Note that when we take the kronecker product of two vectors with the basis vectors $|0\rangle$ and $|1\rangle$ we get a vector with the basis vectors $|00\rangle$, $|01\rangle$, $|10\rangle$ and $|11\rangle$(This is amazing because it allows us to represent every possible state of $n$ qubits with one vector). We can do this with any number of qubits and can represent the state of $n$ qubits using a $2^n$ dimensional vector.
<br><br><br><br><br>

Additionally we can represent the operations on indivigual qubits using matrix multiplication. For example the bitflip operation can be represented as the matrix $\begin{bmatrix}0 & 1 \\ 1 & 0 \end{bmatrix}$. Notice how this maps $|0\rangle$ to $|1\rangle$ and $|1\rangle$ to $|0\rangle$, like how classical bits are flipped. We can operate on the state of two qubits using the kronecker product of two matrices because the kronker product has the nice property of $(A\otimes B) (x \otimes y)=(Ax) \otimes (By)$. This is how we can represent operations on multiple qubits using a single matrix.
<br><br><br><br><br>

To be able to work with qubits in this way we will create a class. I made it easier to take the kroncker product of two qubits by defining the \_\_add\_\_ method to take the kroncker product of two qubits and return a new qubit string representing both of them. I also wrote methods that allow us to apply operations to single qubits within the state using kronecker products of matrices. I called this class qubitstring. 

In [11]:
import numpy as np

GATES = {
    "Hadamard": (1 / np.sqrt(2)) * np.array([[1, 1], [1, -1]]),
    "Identity": np.eye(2),
    "Not": np.array([[0, 1], [1, 0]]),
    "Controlled Phase Shift": lambda k: np.array([[1, 0], [0, np.exp((2 * 1j * np.pi)/ (2**k))]]),
    "Effect 0": np.array([[1, 0], [0, 0]]), 
    "Effect 1": np.array([[0, 0], [0, 1]])  
}


class QubitString:
    def __init__(self, state=None):
        if state is not None:
            self.state = np.array(state, dtype=np.complex128)
            self.num_bits = int(np.log2(len(self.state))) 
        else:
            self.state = np.array([1, 0], dtype=np.complex128)  # Default to |0⟩
            self.num_bits = 1
        self.gates = GATES
    
    def measure(self):
        probabilities = np.abs(self.state) ** 2  # Probabilities of each state
        choice = np.random.choice(len(self.state), p=probabilities)  # Randomly choose a state

        # Collapse the state to chosen output
        self.state = np.zeros_like(self.state, dtype=np.complex128)
        self.state[choice] = 1

        # Return the bitstring of the chosen state
        bitstring = bin(choice)[2:].zfill(self.num_bits)
        return bitstring
        
    def __str__(self):
        return " + ".join(f"{coeff:.3f}|{bin(idx)[2:].zfill(self.num_bits)}⟩" 
                        for idx, coeff in enumerate(self.state) if abs(coeff) > 1e-6)
    
    def __add__(self,other):
        state = np.kron(self.state, other.state)
        combined = QubitString(state=state)
        combined.num_bits = self.num_bits + other.num_bits  # Fix num_bits
        return combined
    
    def apply_gate_to_specific_qubit(self, gate: np.ndarray, index: int = None):
        full_gate = 1
        for i in range(self.num_bits):
            if i == index:
                full_gate = np.kron(full_gate, gate)
            else:
                full_gate = np.kron(full_gate, self.gates["Identity"]) 
        self.state = full_gate @ self.state
    
    def apply_gate_to_all(self, gate: np.ndarray):
        if gate.shape[0]==2:
            for i in range(self.num_bits):
                self.apply_gate_to_specific_qubit(gate, i)
        else:
            self.state = gate @ self.state
    
    def apply_gate_to_range(self, gate: np.ndarray, start: int, end: int):
        for i in range(0,start):
            gate = np.kron(self.gates["Identity"], gate)
        for i in range(end,self.num_bits):
            gate = np.kron(gate, self.gates["Identity"])
        self.state = gate @ self.state

## Deutsh's Algorithm
This is an algorithm that determines wether a function $f: \{0,1\}\rightarrow \{0,1\}$ is balanced(half the inputs map to $1$ the other half map to $0$) or constant(all inputs map to either $0$ or $1$). Classically to solve this you would see if $f(0)\oplus f(1)=0$($\otimes$ denoting the xor operation) and if this statement is true than the function is constant otherwise it is balanced. 
<br><br><br><br>
The quantum version of this algorithm only requires one query to the function to determine wether it is balanced or constant. The algorithm is as follows:
1. Prepare two qubits in the state $|+\rangle|-\rangle$
2. Pass then into the function $f(|x\rangle \otimes |y\rangle)=|x\rangle \otimes |y\oplus f(x)\rangle$
3. Apply a hadamard gate to the first qubit
4. Measure the first qubit
If when you measure the first qubit you get $0$ then the function is constant otherwise it is balanced.



This algorithm blew my mind when I first learned about it because it looks like we are only changing the second qubit, but somehow the first qubit is affected by the function. This is because of someything called phase kickback. I don't understand why from a physics reason phase kickback happens, but I can show you how it works mathematically. The kronecker product has the property that $a \otimes \lambda b = \lambda a \otimes b = \lambda(a \otimes b)$. So if you scale the second qubit, it is the same as scaling the first qubit, or the representation of both qubits. Thus by applying a scalar to the second qubit, we can change the state of the first qubit.

Notice that $f(|+\rang \otimes |-\rang)=|+\rang \otimes \frac{1}{\sqrt{2}}(|0\oplus f(x)\rang - |1 \oplus f(x)\rang)$. Because $0$ is the identity of XOR $0 \oplus f(x)=f(x)$, and because XOR $1$ is the equivalent of a bitflip $1 \oplus f(x) = \overline{f(x)}$ with $\overline{f(x)}$ being the negation of $f(x)$. Thus $|+\rang \otimes \frac{1}{\sqrt{2}}(|0\oplus f(x)\rang - |1 \oplus f(x)\rang)=|+\rang \otimes \frac{1}{\sqrt{2}}(|f(x)\rang - |\overline{ f(x)}\rang)$. 

If you look at the way the value of $f(x)$ rearranges the state, we can see $\frac{1}{\sqrt{2}}(|f(x)\rang - |\overline{ f(x)}\rang)=(-1)^{f(x)}|-\rang$. Thus our state is $|+\rang \otimes (-1)^{f(x)}|-\rang$. This allows us to use the amazing properties of the kronecker product that $a \otimes\lambda b = \lambda a \otimes b$, meaning we can further rewrite our state as $(-1)^{f(x)}|+\rang \otimes |-\rang$. Scaling $|+\rang$ by $(-1)^{f(x)}$ we can see that the state of the two qubits is $\frac{1}{\sqrt{2}}((-1)^{f(0)}|0\rang +(-1)^{f(1)}|1\rang)\otimes |-\rang$. Notice that in this state the first qubit will be in the state $|+\rang$ if $f(0)=f(1)$ and $|-\rang$ if $f(0)\neq f(1)$. Thus by applying the hammard gate to the first qubit and measuring it we can determine wether the function is balanced or constant.

Overall the speedup of this algorithm is not very significant, as instead of querying the function twice we only query it once. But later using the duetsch jorza algorithm we can see that we can get a significant speedup by using quantum computers.

The brief explaination of this code is that if its an injective function it only kicks back once by $\pi$ and if it is a constant function it kicks back by $2\pi$(when both map to $1$) or kicks back by $0$(when everything maps to $0$). Thus because a kickback of $2\pi$ is equivalent to no kickback, we can determine wether the function is constant or injective by checking if the first qubit is unchanged or not.

In [12]:
import numpy as np

#possible functions that map from {0,1} to {0,1}
possible_functions = [lambda x: 0, lambda x: 1, lambda x: x, lambda x: x^1]

# Create the oracle for the given function
def take_function_return_oracle(f):
    oracle = np.eye(2 ** 2)
    
    for i in range(2 ** 2):
        binary_input = bin(i)[2:].zfill(2)[0] 
        if f(int(binary_input)) == 1:
            oracle[i, i] = 0
            oracle[i, i ^ 1] = 1
    return oracle

# Randomly choose a function
f=np.random.choice(possible_functions)


# Create the oracle for the given function
oracle = take_function_return_oracle(f)

def deutsch_algorithm(oracle):

    # Initialize the qubits to 0 and 1
    qubits = QubitString() + QubitString(state=[0, 1])

    # Step 2: Apply Hadamard to both qubits
    hadamard_2q = np.kron(GATES["Hadamard"], GATES["Hadamard"])
    qubits.apply_gate_to_all(hadamard_2q)

    # Step 3: Apply the given oracle function
    qubits.apply_gate_to_all(oracle)

    # Step 4: Apply Hadamard to the first qubit
    qubits.apply_gate_to_specific_qubit(GATES["Hadamard"], 0)

    result = qubits.measure()[0]

    if result == "0":
        return False
    return True

# Running the Deutsch Algorithm
print(f"Function balanced: {deutsch_algorithm(oracle)}")
print(f"Function outputs: f(0)={f(0)}, f(1)={f(1)}")


Function balanced: False
Function outputs: f(0)=1, f(1)=1


## Deutsch-Josza Algorithm
This algorithm is a generalization of Deutsch's algorithm. It determines wether a function $f: \{0,1\}^n \rightarrow \{0,1\}$ is balanced or constant. The algorithm is as follows:
1. Prepare $n+1$ qubits in the state $|+\rangle^{\otimes n}|-\rangle$
2. Pass $|+\rang ^{\otimes n}\otimes |-\rang$ into the function $f(|x\rang \otimes |y\rang)=|x\rang \otimes |y\oplus f(x)\rang$
3. Measure the first $n$ qubits

If the first $n$ qubits are in the state $|0\rang^{\otimes n}$ then the function is constant otherwise it is balanced.

This works pretty much the same as Deutsch's algorithm. The idea being if it is balanced the phase kickback will be nonexistant if the algorithm always maps to $0$ or cancel itself out if it always maps to $1$. But if its balanced the phase kickback will be uneven and the first $n$ qubits will not be in the state $|0\rang^{\otimes n}$.

This is a massive speedup compared to the traditional algorithm. Classically we would have to query the function $2^{n-1}+1$ times to determine wether it is balanced or constant in the worst case. But with a quantum computer we only need to query the function once. Thus the difference in time complexity is $O(2^{n-1}+1)$ vs $O(1)$.

In [13]:
def deutsch_jozsa_algorithm(function, num_bits):
    qubits = QubitString()
    for i in range(num_bits-1):
        qubits = qubits + QubitString()
    qubits = qubits + QubitString(state=[0, 1])

    qubits.apply_gate_to_all(GATES["Hadamard"])

    # Apply the given oracle function
    qubits.apply_gate_to_all(function)

    qubits.apply_gate_to_all(GATES["Hadamard"])
    for bit in qubits.measure()[:-1]:
        if bit != "0":
            return True
    return False

num_bits = 3
#generate two different random indices
ind1=np.random.randint(0, num_bits)
while True:
    ind2=np.random.randint(0, num_bits)
    if ind1!=ind2:
        break



print(ind1)
possible_functions = [lambda bits: 0, lambda bits: 1, lambda bits: int(bits[ind1])^int(bits[ind2]), lambda bits: (int(bits[ind1]) ^ int(bits[ind2]))^1, lambda bits: int(bits[ind1]), lambda bits: int(bits[ind1])^1]

def take_function_return_oracle(f):
    oracle = np.eye(2 ** (num_bits + 1))

    for i in range(2 ** (num_bits + 1)):
        binary_input = bin(i)[2:].zfill(num_bits + 1)[:-1]
        if f(binary_input) == 1:
            oracle[i, i] = 0
            oracle[i, i ^ 1] = 1
    return oracle

f = np.random.choice(possible_functions)

oracle=take_function_return_oracle(f)
print(f"Function balanced: {deutsch_jozsa_algorithm(oracle, num_bits)}")
for i in range(2 ** num_bits):
    binary_input = bin(i)[2:].zfill(num_bits)
    print(f"f({binary_input})={f(binary_input)}")

2
Function balanced: True
f(000)=1
f(001)=0
f(010)=1
f(011)=0
f(100)=0
f(101)=1
f(110)=0
f(111)=1


## Quantum Fourier Transform
We have worked a lot with qubits and their phases through phase kickback, but we don't really have a way to incorporate the phase of a qubit into the measured state of a qubit other than changing it from $|-\rang$ to $|+\rang$ and vice versa. The quantum fourier transform is a way to incorporate the phase of a qubits into its amplitudes. 

The quantum fourier transform as a function looks like this: $\mathrm{QFT}(|x\rangle) = \frac{1}{\sqrt{2^n}} \sum_{k=0}^{2^n - 1} e^{2\pi i xk / 2^n} |k\rangle$

This function definition looks terrifying but its easier to understand when we look at it in terms of basis vectors and a change of basis. If we apply the quantum fourier transform to the set $\{|0\rang, |1\rang\}$ it maps it to the set $\{|+\rang, |-\rang\}$ and then if we apply the QFT again it maps it to the set $\{|0\rang, |1\rang\}$.  This is how it incorporates the phase of a qubit into its amplitude. 

In [14]:
from functools import reduce

#nicely combines the kronecker product of a list of matrices
def kron_iterative(matrices):
    return reduce(np.kron, matrices)

# Put single qubit gate into a larger unitary matrix
def embed_single_qubit_gate(gate, target, n):
    matrices = [gate if i == target else np.eye(2) for i in range(n)]
    return kron_iterative(matrices)

# put controlled phase gate into a larger unitary matrix
def embed_controlled_phase_gate(k, control, target, n):
    phase = np.exp(2j * np.pi / (2 ** k))
    N = 2 ** n
    U = np.eye(N, dtype=complex)
    for i in range(N):
        bits = [int(x) for x in format(i, f'0{n}b')]
        if bits[control] == 1 and bits[target] == 1:
            U[i, i] = phase
    return U

# Put swap gate into a larger unitary matrix
def embed_swap_gate(i, j, n):
    N = 2 ** n
    U = np.zeros((N, N), dtype=complex)
    for a in range(N):
        bits = list(format(a, f'0{n}b'))
        bits[i], bits[j] = bits[j], bits[i]
        b = int(''.join(bits), 2)
        U[b, a] = 1
    return U

# Create the quantum Fourier transform operator
def qft_operator(n):
    N = 2 ** n
    U = np.eye(N, dtype=complex)
    for j in range(n):
        U = embed_single_qubit_gate(GATES["Hadamard"], j, n) @ U
        for k in range(j + 1, n):
            U = embed_controlled_phase_gate(k - j + 1, j, k, n) @ U
    for j in range(n // 2):
        U = embed_swap_gate(j, n - j - 1, n) @ U
    return U

def qft_multiqubit(qubit: QubitString):
    U = qft_operator(qubit.num_bits)
    qubit.state = U @ qubit.state

def inv_qft_multiqubit(qubit: QubitString):
    U = qft_operator(qubit.num_bits)
    U = np.conj(U).T
    qubit.state = U @ qubit.state

qubit = QubitString()
for i in range(1):
    qubit = qubit + QubitString()

print(qubit)
qft_multiqubit(qubit)
print(qubit)
inv_qft_multiqubit(qubit)
print(qubit)

1.000+0.000j|00⟩
0.500+0.000j|00⟩ + 0.500+0.000j|01⟩ + 0.500+0.000j|10⟩ + 0.500+0.000j|11⟩
1.000+0.000j|00⟩


## Quantum Phase Estimation
This is an algorithm that given an eigenvector $v$ and a unitary matrix $U$ this algorithm finds the eigenvalue of $v$. Note that since $U$ is unitary the eigenvalues of $U$ are of the form $e^{2i\pi\theta}$, where $\theta$ is a real number. So to simplify this problem rather than finding the eigenvalue we will find $\theta \in [0,1]$, note that there are other possible values of $\theta$ but by narrowing our range there will be a unique value of $\theta$. The algorithm specifically for the 2x2 matrix case is as follows:
1. Prepare $n$ precision qubits in the state $|0\rang^{\otimes n}$ where $n$ is the bits of precision you want
2. Prepare a qubit in the state $|v\rang$
3. Apply the Hadamard gate to the first $n$ qubits
4. Set up a control gate for each precision qubit $i$ that applies $U^{2^i}$ to the qubit |v\rang
5. Apply the inverse fourier transform to the first $n$ qubits
6. Measure the first $n$ qubits
7. The result is the binary representation of $\theta$ with $n$ bits of precision

This algorithm is another case where phase kick explains it all. Whenever we apply the unitary matrix $U^n$ to the qubit $|v\rang$ we it becomes $e^{2i\pi\theta n}|v\rang$. By applying $U^1$ whenever the first control bit is $1$, $U^2$ whenever the second control bit is $1$, $U^4$ when the third control bit is $1$ and so on we can see that the state of $|x_n...x_1\rang$ after applying all of these controled gates is $e^{2 i\pi \theta x_n...x_1}|x_n...x_1\rang$ where $x_n...x_1$ is a number represented in binary. Thus if we have $n$ precision bits our final state is $|0...0\rang +E^{2 i \pi \theta n}|0...1\rang+...+E^{2 i \pi \theta n}|1...1\rang$ which when we apply a inverse fourier transformation, which then places our state into the form $|\theta\rang$ which then we measure as $\theta$. 

In [15]:
def controlled_unitary(U, control_index, num_control_qubits, n_eigen, power=1):
    #raise U to the proper power 
    U_power = np.linalg.matrix_power(U, power)

    #create our matrix we are going to return
    total_qubits = num_control_qubits + n_eigen
    N = 2**total_qubits
    C = np.zeros((N, N), dtype=complex)

    #go through the size of the matrix
    for i in range(N):
        #get the bits of the current index
        bits = list(format(i, f'0{total_qubits}b'))

        #if the control bit is 0 we just put the identity matrix 
        if bits[control_index] == '0':
            C[i, i] = 1
        else:
            eigen_index = int(''.join(bits[num_control_qubits:]), 2)
            for j in range(2**n_eigen):
                new_bits = bits[:num_control_qubits] + list(format(j, f'0{n_eigen}b'))
                j_index = int(''.join(new_bits), 2)
                C[j_index, i] = U_power[j, eigen_index]
    return C

def inv_qft_on_control_operator(num_control_qubits, total_qubits):
    QFT = qft_operator(num_control_qubits)  
    QFT_inv = np.conj(QFT).T
    eigen_dim = 2**(total_qubits - num_control_qubits)
    return np.kron(QFT_inv, np.eye(eigen_dim, dtype=complex))

def measure_subsystem(qubit: QubitString, subsystem_qubits: list):
    num_sub = len(subsystem_qubits)
    total = qubit.num_bits
    n_rest = total - num_sub
    probs = []
    outcomes = []
    for i in range(2**num_sub):
        p = 0
        control_bits = format(i, f'0{num_sub}b')
        for j in range(2**n_rest):
            eigen_bits = format(j, f'0{n_rest}b')
            full_index = int(control_bits + eigen_bits, 2)
            p += np.abs(qubit.state[full_index])**2
        probs.append(p)
        outcomes.append(control_bits)
    probs = np.array(probs)
    probs = probs / np.sum(probs)
    measured = np.random.choice(outcomes, p=probs)
    return measured


def phase_estimation_algorithm(U, eigenstate: QubitString, num_control_qubits: int):

    #set up control bits and eigenstate
    control = QubitString()
    for _ in range(num_control_qubits - 1):
        control = control + QubitString(state=[1, 0])
    combined = control + eigenstate
    total_qubits = combined.num_bits
    n_eigen = eigenstate.num_bits

    #place the control bits into a super position 
    for i in range(num_control_qubits):
        combined.apply_gate_to_specific_qubit(GATES["Hadamard"], i)

    #applying U to some power to the bit storing our eigenstate
    for i in range(num_control_qubits):
        power = 2 ** (num_control_qubits - i - 1)
        CU = controlled_unitary(U, control_index=i, num_control_qubits=num_control_qubits, n_eigen=n_eigen, power=power)
        combined.state = CU @ combined.state

    #apply the inverse QFT to the control qubits NOT THE EIGENSTATE
    inv_qft_control = inv_qft_on_control_operator(num_control_qubits, total_qubits)
    answer = QubitString(state=inv_qft_control @ combined.state).measure()[:-1]

    estimated_phase = int(answer, 2) / (2 ** num_control_qubits)
    return estimated_phase

#determines the eigenvalue of the unitary matrix U
phi = 0.304

#the matrix U that has an eigenvector |1⟩ and eigenvalue e^(2*pi*i*phi)
U = np.array([[1, 0],
              [0, np.exp(2j * np.pi * phi)]])


#the eigenvector |1⟩
eigen = QubitString(state=[0, 1])

#precision of the phase estimation algorithm
num_control_qubits = 9

#run the actual algorithm 

estimated_phi = phase_estimation_algorithm(U, eigen, num_control_qubits)
print("Estimated phase:", estimated_phi)


Estimated phase: 0.3046875


Unfortunately even though our previous algorithm for phase estimation would be incredibly efficent with a quantum computer, it is not very practical. For this reason we will be using numpy to calculate the eigenvalues of a matrix. This will allow us to move on to the next algorithm which is shor's algorithm. 

In [16]:
def phase_estimation_algorithm(U, eigenstate, num_control_qubits):
    """
    Classical simulation of phase estimation:
    - Find eigenvalues of U
    - Randomly pick one eigenvalue weighted by overlap with eigenstate
    - Extract phase from eigenvalue λ = exp(2πi φ)
    """
    eigenvalues, eigenvectors = eig(U)
    # Compute overlap |<eigenvector|eigenstate>|^2 for weighting
    overlaps = np.abs(eigenvectors.conj().T @ eigenstate.state)**2
    overlaps /= overlaps.sum()  # normalize

    # Choose an eigenvalue according to overlap probabilities
    idx = np.random.choice(len(eigenvalues), p=overlaps)
    lam = eigenvalues[idx]
    phi = np.angle(lam) / (2 * np.pi)  # get phase in [-0.5, 0.5]
    if phi < 0:
        phi += 1  # map to [0, 1)
    return phi

## Shor's Algorithm 
This is an algorithm that finds the prime factors of a number $N$.

It works by picking a random number $a$ such that $1 < a < N$. If $a$ and $N$ are not coprime than we can find a factor of $N$ by taking the gcd of $a$ and $N$. If they are coprime then we can use the fact that if $|a|$ is even and $(a^{|a|/2}+1)(a^{|a|/2}-1) \equiv 0 \mod N$. This means that $N$ divides the product of $(a^{|a|/2}+1)$ and $(a^{|a|/2}-1)$ but not them individually. Thus the GCD of $N$ and $(a^{|a|/2}+1)$ or $(a^{|a|/2}-1)$ will give us a factor of $N$.

The quantum speedup comes from using phase estimation to find the order of $a$ modulo $N$. 

The failures of this algorithm come from the fact that the period of $a$ might be odd, or $(a^{|a|/2})\equiv -1 \mod N$ because then the expression $(a^{|a|/2}+1)(a^{|a|/2}-1)$ will not be non-trivial factors of $N$.

In [27]:
from math import gcd, ceil, log2, sqrt
import random
import numpy as np
from fractions import Fraction
import numpy as np
from numpy.linalg import eig
from math import gcd, ceil, log2, sqrt
import random
from fractions import Fraction

# Modular exponentiation padded to power-of-two dimension
def mod_exp_unitary(x, N):
    n_eigen = int(ceil(log2(N)))
    dim = 2**n_eigen
    U = np.eye(dim, dtype=complex)
    for y in range(N):
        target = (x * y) % N
        U[target, y] = 1
        U[y, y] = 0
    return U

# Refined continued fraction method
def refined_continued_fraction(phi, N):
    for r in range(2, N):
        s = round(r * phi)
        if abs(phi - s / r) < 1 / (2 * r * r):
            return s, r
    frac = Fraction(phi).limit_denominator(N)
    return frac.numerator, frac.denominator

# Perform multiple trials and pick the best phase estimate
def repeated_phase_estimation(U, eigenstate, num_control_qubits, N, trials=3):
    estimates = []
    for _ in range(trials):
        phi = phase_estimation_algorithm(U, eigenstate, num_control_qubits) % 1
        _, r = refined_continued_fraction(phi, N)
        estimates.append((phi, r))
    # Return the phase with the smallest r
    return min(estimates, key=lambda x: x[1])

# Full Shor's algorithm
def shors_algorithm(N, num_control_qubits=9, phase_trials=3):
    possible_x = [x for x in range(2, N) if gcd(x, N) == 1]
    random.shuffle(possible_x)

    for attempt, x in enumerate(possible_x, start=1):
        print(f"[Attempt {attempt}] Chosen x = {x}")

        U = mod_exp_unitary(x, N)
        n_eigen = int(ceil(log2(N)))
        
        # Uniform superposition eigenstate
        eigen_vec = np.array([1/sqrt(N) if i < N else 0 for i in range(2**n_eigen)], dtype=complex)
        eigenstate = QubitString(state=eigen_vec)

        phi, r = repeated_phase_estimation(U, eigenstate, num_control_qubits, N, trials=phase_trials)
        print(f"  Raw phase = {phi}, estimated period r = {r}")

        # Reject if not even
        if r % 2 != 0:
            print("  r is odd, continuing...\n")
            continue

        # Check for false period
        if pow(x, r, N) != 1:
            print("  False period: x^r mod N ≠ 1, continuing...\n")
            continue

        plus = pow(x, r//2, N) + 1
        minus = pow(x, r//2, N) - 1
        f1, f2 = gcd(plus, N), gcd(minus, N)

        if f1 in (1, N) or f2 in (1, N):
            print("  Trivial factors, continuing...\n")
            continue

        print(f"  Success: factors = ({f1}, {f2})\n")
        return f1, f2

    print("No non-trivial factors found.")
    return None

# Example run
num = 85  
factors = shors_algorithm(num)
print("Factors of "+str(num)+":", factors)



[Attempt 1] Chosen x = 12
  Raw phase = 0.0, estimated period r = 2
  False period: x^r mod N ≠ 1, continuing...

[Attempt 2] Chosen x = 2
  Raw phase = 0.0, estimated period r = 2
  False period: x^r mod N ≠ 1, continuing...

[Attempt 3] Chosen x = 47
  Raw phase = 0.0, estimated period r = 2
  False period: x^r mod N ≠ 1, continuing...

[Attempt 4] Chosen x = 38
  Raw phase = 0.0, estimated period r = 2
  False period: x^r mod N ≠ 1, continuing...

[Attempt 5] Chosen x = 23
  Raw phase = 0.0, estimated period r = 2
  False period: x^r mod N ≠ 1, continuing...

[Attempt 6] Chosen x = 83
  Raw phase = 0.0, estimated period r = 2
  False period: x^r mod N ≠ 1, continuing...

[Attempt 7] Chosen x = 59
  Raw phase = 0.0, estimated period r = 2
  False period: x^r mod N ≠ 1, continuing...

[Attempt 8] Chosen x = 44
  Raw phase = 0.0, estimated period r = 2
  False period: x^r mod N ≠ 1, continuing...

[Attempt 9] Chosen x = 8
  Raw phase = 0.0, estimated period r = 2
  False period: x^r mo